In [ ]:
import pandas as pd
import openai
import time
from openai.embeddings_utils import get_embedding
from openai.embeddings_utils import cosine_similarity
openai.api_key = "put you api key. if you don't have API key, pass."

# Ex2. Playing Spyfall with GPTs

In [ ]:
rule = '''
The rules for a game of "Spyfall" are as follows:

There are five players in total, each with a number from 1 to 5. One of them is chosen to be the 'spy', and the goal of the remaining players is to find the spy.

The game is played over a total of 10 turns, each turn consisting of
1. a player asks another player a question
1. the player who was asked the question answers
1. after hearing the question and answer, decide if you want to point to a specific player as the spy
1.1. vote on whether to hang the player (if someone has been named)
1. Decide if the spy will reveal their identity This is done in this order.
The first player to ask a question is Player 1, and so on until the player who was asked the question has the opportunity to ask a question on their next turn.

Each player has the opportunity to name another player as the "Spy" only once during the game. Once a player is named, the rest of the players vote on whether that player should be hung. If the player receives 4 or more votes, they are hung. If the hanged player was a spy, the not-spy players win; otherwise, the spy wins.

If the spy reveals that they are the spy, the spy must guess the location. If the location is correct, the spy wins; if it is incorrect, the spy loses.

If all 10 turns have been played and the spy has not been revealed, all players vote for the player they think is the spy. The player with the most votes is assumed to be the spy, and if this player is indeed the spy, the normal players win, otherwise the spy wins. If there is more than one player with the most votes, the vote is redone.

As the game progresses, at each turn, players will receive the following requests. (There is nothing players can do to affect the game other than the requests below, so players will need to play with that in mind.

1. to the player asking the question (for the first turn, Player 1; thereafter, the player who answered the question on the previous turn):
    1. Now it's your turn to ask a question. Choose 1-5 players other than yourself, and write a question.
    Please write your response in the form (n_player,"question")
2. When asking for answers to subsequent questions
    1. In the nth question, player P asked you the following. Please write your answer to this question inside the "" mark.'
3. After all questions and answers have been heard, all players will be asked the following on each turn. Note that players may not interact with each other other other than to name players or not to name players.
    1. At this point, do you want to use the opportunity to name a spy? If yes, write the number of the player you want to name, otherwise write an X in your response.Please only write numbers or Xs in your response.
4. if a player has named someone else as a player, they will be asked to vote as follows. Note that players cannot interact with each other other other than O and X. 1.
    1. 'Subsequently, player P accused player Q of being a spy. Therefore, a vote was held. If you want to hang player Q as a spy, please enter O. If not, please enter X.'
5. At this point, if no one has accused another player this turn, or if a vote is held but does not receive a sufficient number of votes, the game continues and the spy reveals his or her identity and decides whether to guess the location as follows.
    1. At this point, would you identify yourself as a spy and guess the place?
    If you would like to reveal your identity and guess the place, please enter the name of the place. If not, please enter an X. Please only answer with a place name or X.
6. If the spy has not revealed his or her identity at this point, you will move on to the next turn and repeat the sequence from 1.
7. At the end of turn 10, a vote is taken to finally reveal the spy. Note that requests are received in order, starting with Player 1, and may include a three-sentence statement along with the person to be voted out. This vote is repeated until a particular player receives a majority of the votes.
Additional. The location is chosen as one of the following lists (no capitalization)
[airplane, amusement park, bank, beach, carnival, casino, circus tent, corporate party, crusader army, day spa, embassy, hospital, hotel, military base, movie studio, nightclub, ocean liner, passenger train, pirate ship, polar station, police station, restaurant, school, service station, space station, submarine, supermarket, theater, university, zoo]
'''
strategy = '''
(Additional) When it comes to strategy and how to play "Spy Fall," the following facts can be helpful.
1. players should basically keep their conversations vague so that the spy doesn't notice the location. You can execute this effectively by framing your responses to correspond to at least 4 or 5, and, depending on the level of ambiguity needed, up to 20~30 different locations.
2. spies will also try to prove that they are not spies by choosing their questions in this way.
3. You'll also want to confuse the spy as much as possible in order to reach your goal before they do. Keep in mind that unlike normal players, spies don't have information about the location. 
4. In response, the Spy will try to deflect suspicion away from themselves by using phrases that imply specific information about the location, or by pointing to other players to add to the confusion.
5. To find the spy through the game, you need to closely observe the questions and answers, as well as the nominations and votes of other players. Observe who is suspecting whom.
6. In the game, players have no way to express their opinions other than by asking questions, answering questions, and calling out spies. Therefore, if you suspect a player, you should immediately call them out as a spy. If this is a reasonable suspicion, the other players will agree, and if not, they will disagree. Calling out spies is an important strategy because it is the only way to know what other players are doing.
7. Spies should reveal their identity when they have enough information about the location, or when they believe the risk of other players identifying them as a spy is sufficiently great.
'''
rule + strategy

# game start
gamestart = '''
The game has started.
'''
youspy = '''You're a spy.

'''
notspy = '''
You're not a spy; the location is '{}'.

'''
youare = '''
You are player {}.
'''

questioner = ''' Now it's your turn to ask a question. Choose 1-5 players other than yourself, and write a question.
 Please write your response in the form (n_player,"question")'''
q4 = '''In the {}th question, player {} asked player {} the following.'''
q2you = '''In the {}th question, player {} asked you the following.'''
askanswer = 'Please write your answer to this question inside the "" mark.'

answerwas = ''' To this question, player {} responded with the following.'''

whetheraccuse = '''At this point, do you want to use the opportunity to name a spy? If yes, write the number of the player you want to name, otherwise write an X in your response.Please only write numbers or Xs in your response.'''
nooneaccused = ''' At the end of this question, no one accused anyone else of being a spy.'''
whethercomeout = '''At this point, would you identify yourself as a spy and guess the place?
If you would like to reveal your identity and guess the place, please enter the name of the place. If not, please enter an X. Please only answer with a place name or X..'''
notcameout = ''' The spy did not reveal their identity.'''

# accusation
theyaccused = 'Subsequently, player {} accused player {} of being a spy.'
youagree = 'Therefore, a vote was held. If you want to hang player {} as a spy, please enter O. If not, please enter X.'

Cf. when running a game by GPT-4 api, average cost of 1 game is around USD 2.5-3. It varies on how the spy plays game well, and if the game ends later, the cost would be USD 4-5.

In [ ]:
RSU = rule+strategy+youare

### Random seeds for spy

In [ ]:
import random

random.seed(0)

# repeat [1, 2, 3, 4, 5] 6 times to get the same number of occurrences
numbers = [1, 2, 3, 4, 5] * 6

# randomly shuffle the order of the list
random.shuffle(numbers)

print(numbers)

In [ ]:
import ast
import pyperclip
import datetime as dt
import random

# we set the number of gamecode as the number of the location list.
# You can modify this option.

num = 9
gamecode = 'ena100{}'.format(num+1)
game_start = dt.datetime.now()

turn = 1
#turnlog = {}
gamelog = ''
questions = []
answers = []
p_que = 1

location_list = ['airplane', 'amusement park', 'bank', 'beach', 'carnival',
                 'casino', 'circus tent', 'corporate party',
                 'crusader army', 'day spa', 'embassy', 'hospital', 'hotel',
                 'military base', 'movie studio', 'nightclub',
                 'ocean liner', 'passenger train', 'pirate ship', 
                 'polar station', 'police station', 'restaurant', 'school',
                 'service station', 'space station', 'submarine', 'supermarket',
                 'theater', 'university', 'zoo']



spy = numbers[num]
location =location_list[num]

RSU = rule+strategy+youare
accusechance = {p: True for p in range(1, 6)}
gameover = None
OX = {} #related to accuselog - whether they accoused other player.

# for logging the vote
votelog = []

## Request response to GPT

In [ ]:
def getGPT4response(script):
    return openai.ChatCompletion.create(
            model="gpt-4",
            messages=[
                {"role":"system","content":"Be sure to respond as directed by the user."},
                {"role": "user", "content": script}
            ]
        )['choices'][0]["message"]["content"]

def q_getGPT4response(script):
    return openai.ChatCompletion.create(
            model="gpt-4",
            messages=[
                
                {"role": "user", "content": script},
                {"role":"assistant","content":"(p,'exampleresponse')"},
            ]
        )['choices'][0]["message"]["content"]

def a_getGPT4response(script):
    return openai.ChatCompletion.create(
            model="gpt-4",
            messages=[
                
                {"role": "user", "content": script},
                {"role":"assistant","content":'"exampleresponse"'},
            ]
        )['choices'][0]["message"]["content"]

def accuse_getGPT4response(script):
    return openai.ChatCompletion.create(
            model="gpt-4",
            messages=[
                {"role":"assistant","content":'X'},
                {"role":"assistant","content":'1'},
                {"role":"assistant","content":'2'},
                {"role":"assistant","content":'3'},
                {"role":"assistant","content":'4'},                
                {"role":"assistant","content":'5'},
                {"role": "user", "content": script},

            ]
        )['choices'][0]["message"]["content"]

def vote_getGPT4response(script):
    return openai.ChatCompletion.create(
            model="gpt-4",
            messages=[
                {"role":"assistant","content":'O'},
                {"role":"assistant","content":'X'},   
                {"role": "user", "content": script},

            ]
        )['choices'][0]["message"]["content"]



def co_getGPT4response(script):
    return openai.ChatCompletion.create(
            model="gpt-4",
            messages=[
                {"role":"assistant","content":'X'},
                {"role":"assistant","content":'location name'},
                {"role": "user", "content": script},

            ]
        )['choices'][0]["message"]["content"]


# How about 3.5?

We provide the same code but runs with gpt-3.5-turbo. We've also tried to reduce computing cost and tested various ways to circumvent the cost problem with GPT-3.5-turbo. However, as you read in the main text, we failed.


In [ ]:
def getGPT35response(script):
    return openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role":"system","content":"Be sure to respond as directed by the user."},
                {"role": "user", "content": script}
            ]
        )['choices'][0]["message"]["content"]

def q_getGPT35response(script):
    return openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                
                {"role": "user", "content": script},
                {"role":"assistant","content":"(p,'exampleresponse')"},
            ]
        )['choices'][0]["message"]["content"]

def a_getGPT35response(script):
    return openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                
                {"role": "user", "content": script},
                {"role":"assistant","content":'"exampleresponse"'},
            ]
        )['choices'][0]["message"]["content"]

def accuse_getGPT35response(script):
    return openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role":"assistant","content":'X'},
                {"role":"assistant","content":'1'},
                {"role":"assistant","content":'2'},
                {"role":"assistant","content":'3'},
                {"role":"assistant","content":'4'},                
                {"role":"assistant","content":'5'},
                {"role": "user", "content": script},


            ]
        )['choices'][0]["message"]["content"]

def vote_getGPT35response(script):
    return openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role":"assistant","content":'O'},
                {"role":"assistant","content":'X'},  
                {"role": "user", "content": script},


            ]
        )['choices'][0]["message"]["content"]



def co_getGPT35response(script):
    return openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role":"assistant","content":'X'},
                {"role":"assistant","content":'location name'},
                {"role": "user", "content": script},

            ]
        )['choices'][0]["message"]["content"]


## Iteration starts

From the start of the game, you just run the every code below repetitively - until the game ends.

### Question

In [ ]:
# question
samepart = gamelog+questioner
if p_que != spy:
    script = RSU.format(p_que)+notspy.format(location)+samepart
    pyperclip.copy(script)
    questions.append(ast.literal_eval(q_getGPT4response(script)))
elif p_que == spy:
    script = RSU.format(p_que)+youspy+samepart
    pyperclip.copy(script)
    questions.append(ast.literal_eval(q_getGPT4response(script)))
print(questions[-1])

### Answer

In [ ]:
# answer
p_ans = questions[turn-1][0]
samepart = gamelog+q2you.format(turn, p_que)+questions[turn-1][1]+askanswer
if p_ans != spy:
    script = RSU.format(p_ans)+notspy.format(location)+samepart
    pyperclip.copy(script)
    answers.append(a_getGPT4response(script))
if p_ans == spy:
    script = RSU.format(p_ans)+youspy+samepart
    pyperclip.copy(script)
    answers.append(a_getGPT4response(script))
print(answers[-1])

### Accusation

In [ ]:
# wherther to accuse
gamelog = gamelog + q4.format(turn, p_que, p_ans) + \
    questions[turn-1][1]+answerwas.format(p_ans)+answers[turn-1]


In [ ]:
OX[turn] = {}

In [ ]:
# Replace the ones who don't have right to accuse
#(who already accused someone earlier)

whocanaccuse = [p for p, v in accusechance.items() if v is True]
print(whocanaccuse)
for p in whocanaccuse:
    if p not in OX[turn].keys():
        if p != spy:
            script = RSU.format(p)+notspy.format(location) + \
                gamelog+whetheraccuse
            pyperclip.copy(script)
            OX[turn][p] = accuse_getGPT4response(script)
        elif p == spy:
            script = RSU.format(spy)+youspy+gamelog + whetheraccuse
            pyperclip.copy(script)
            OX[turn][p] = accuse_getGPT4response(script)
        time.sleep(5)
        print(p)
print(OX[turn])

### Processing accusation

In [ ]:
paccusedp = 'Player {} has accused player {} of being a spy.'
sostartvote =  'Therefore, we will vote on each player in turn.'
allagreed =  '''Four or more players have decided to hang player {}. (Voting result: {})'''
notallagreed = notallagreed = 'A vote was started by player {} to hang player {}, but the vote was defeated. (Vote result: {})'

# processing accusation
p_hang = {}
whocanaccuse = [p for p, v in accusechance.items() if v is True]

# Adding gamelog "who picked whom"
someoneaccused = False
for p in whocanaccuse:
    if OX[turn][p] != 'X':
        someoneaccused = True
        gamelog += paccusedp.format(p, OX[turn][p])
if someoneaccused == True:
    gamelog += sostartvote
else:
    gamelog += nooneaccused


# Start Accusation
for p in whocanaccuse:
    # Pass if no one accused anyone
    if OX[turn][p] == 'X':
        print('nooneaccused')
        pass
    else:
        try:
            p_hang[p] = int(OX[turn][p])
        except:
            continue
        p_hung = p_hang[p]
        accusechance[p] = False
        # 투표 들어감
        votes = {q: None for q in range(1, 6)}
        for q in range(1, 6):
            if q == p:
                votes[q] = 'O'  # 고발한 사람은 자동 O
            elif q == p_hung:
                votes[q] = 'X'  # 고발당한 사람은 자동 X
            else:
                if q != spy:
                    script = (RSU.format(p)
                              + notspy.format(location)
                              + gamelog
                              + theyaccused.format(p, p_hung)
                              + youagree.format(p_hung))
                    pyperclip.copy(script)
                    votes[q] = getGPT4response(script)
                else:
                    script = (RSU.format(spy)
                              + youspy
                              + gamelog
                              + theyaccused.format(p_hang[p], p_hung)
                              + youagree.format(p_hung))
                    pyperclip.copy(script)
                    votes[q] = vote_getGPT4response(script)
            time.sleep(5)
        # 개표
        xcount = 0
        for q in range(1, 6):
            if votes[q] == 'O':
                pass
            elif votes[q] == 'X':
                xcount += 1
            else:
                pass
        # 결과 확인
        if xcount < 2:
            gameover = allagreed.format(
                p_hung, votes)
            if p_hung == spy:
                gameover += 'Player {} is the spy. Spy loses. '.format(spy)
                gamelog += gameover
                spywin = False
            else:
                gameover += 'Player {} is not the spy.'.format(p_hung)
                gameover += 'The spy is player {}. The spy wins'.format(spy)
                gamelog += gameover
                spywin = True
            # 게임 끝나기 전에도 votelog 넣어야됨.
            votelog.append({'turn': turn,
                            'hang': p,
                            'hung': p_hung,
                            'result': votes,
                            'xcount': xcount})
            break
        else:
            gamelog += notallagreed.format(
                p, p_hung, votes)

    if OX[turn][p] != 'X':
        # Add vote log at the end of vote procedure
        votelog.append({'turn': turn,
                        'hang': p,
                        'hung': p_hung,
                        'result': votes,
                        'xcount': xcount})

### For spy: whether to reveal themselves

In [ ]:
#reveal?
if gameover == None:
    script = RSU.format(spy)+youspy+gamelog+whethercomeout
    pyperclip.copy(script)
    spycomeout = getGPT4response(script)
    if spycomeout == 'X':
        pass
    elif spycomeout == location:
        gameover = 'Player {} reveals that they are the spy, and they guessed the location. The spy wins.'.format(spy)
        gamelog += gameover
        spywin = True
    else:
        gameover = 'Player {} identified themself as the spy and said the correct answer was {}, but it was {}, so the spy loses..'.format(spy,spycomeout,location)
        gamelog += gameover
        spywin = False
else:
    print('game already over:{}'.format(gameover))
print(gamelog)

### Check if game ends - or go to the next turn.

In [ ]:
if gameover != None:
    print(gameover)
    full_game_data = {'gamecode': gamecode,
                        'spy': spy,
                      'location': location,
                      'spywin': spywin,
                      'game_start': game_start,
                      'game_end': dt.datetime.now(),
                      'gamelog': gamelog,
                      'votelog': votelog,
                      'total_turn': turn,
                      'questions': questions,
                      'answers': answers,
                      'accuselog': OX
                      }

else:
    gamelog += notcameout
    turn += 1
    p_que = questions[turn-2][0]
    print("turn {} start".format(turn))

## Now go up to the "Iterationg starts"

# Final votes

In [ ]:
# Start vote procedure
for p in whocanaccuse:
    # if no one accused, pass
    if OX[turn][p] == 'X':
        print('nooneaccused')
        pass
    else:
        try:
            p_hang[p] = int(OX[turn][p])
        except:
            continue
        p_hung = p_hang[p]
        accusechance[p] = False
        # Start Voting
        votes = {q: None for q in range(1, 6)}
        for q in range(1, 6):
            if q == p:
                votes[q] = 'O'  # Automatically 'O' for someone who start voting
            elif q == p_hung:
                votes[q] = 'X'  # Automatically 'X' for someone who get accused
            else:
                if q != spy:
                    script = (RSU.format(p)
                              + notspy.format(location)
                              + gamelog
                              + theyaccused.format(p, p_hung)
                              + youagree.format(p_hung))
                    pyperclip.copy(script)
                    votes[q] = getGPT4response(script)
                else:
                    script = (RSU.format(spy)
                              + youspy
                              + gamelog
                              + theyaccused.format(p_hang[p], p_hung)
                              + youagree.format(p_hung))
                    pyperclip.copy(script)
                    votes[q] = vote_getGPT4response(script)
            time.sleep(5)
        # 개표
        xcount = 0
        for q in range(1, 6):
            if votes[q] == 'O':
                pass
            elif votes[q] == 'X':
                xcount += 1
            else:
                pass
        # 결과 확인
        if xcount < 2:
            gameover = allagreed.format(
                p_hung, votes)
            if p_hung == spy:
                gameover += 'Player {} is the spy. Spy loses. '.format(spy)
                gamelog += gameover
                spywin = False
            else:
                gameover += 'Player {} is not the spy.'.format(p_hung)
                gameover += 'The spy is player {}. The spy wins'.format(spy)
                gamelog += gameover
                spywin = True
            # Append votelog before the game ends.
            votelog.append({'turn': turn,
                            'hang': p,
                            'hung': p_hung,
                            'result': votes,
                            'xcount': xcount})
            break
        else:
            gamelog += notallagreed.format(
                p, p_hung, votes)

    if OX[turn][p] != 'X':
        # Append votelog after the vote ends -> subsequently start another vote
        votelog.append({'turn': turn,
                        'hang': p,
                        'hung': p_hung,
                        'result': votes,
                        'xcount': xcount})

# Saving games

## Appending new game data on full data

In [ ]:
dataset[full_game_data['gamecode']] = full_game_data

### show former game data

In [ ]:
dataset

### Make a pickle

In [ ]:
import pickle

In [ ]:
with open('231102spyfalldataset.pkl','wb') as file:
    pickle.dump(dataset,file)

# Ex 1. First question comparison: GPT-4 vs GPT-3.5-turbo

In [ ]:
testscript = '''
The rules for a game of "Spyfall" are as follows:

There are five players in total, each with a number from 1 to 5. One of them is chosen to be the 'spy', and the goal of the remaining players is to find the spy.

The game is played over a total of 10 turns, each turn consisting of
1. a player asks another player a question
1. the player who was asked the question answers
1. after hearing the question and answer, decide if you want to point to a specific player as the spy
1.1. vote on whether to hang the player (if someone has been named)
1. Decide if the spy will reveal their identity This is done in this order.
The first player to ask a question is Player 1, and so on until the player who was asked the question has the opportunity to ask a question on their next turn.

Each player has the opportunity to name another player as the "Spy" only once during the game. Once a player is named, the rest of the players vote on whether that player should be hung. If the player receives 4 or more votes, they are hung. If the hanged player was a spy, the not-spy players win; otherwise, the spy wins.

If the spy reveals that they are the spy, the spy must guess the location. If the location is correct, the spy wins; if it is incorrect, the spy loses.

If all 10 turns have been played and the spy has not been revealed, all players vote for the player they think is the spy. The player with the most votes is assumed to be the spy, and if this player is indeed the spy, the normal players win, otherwise the spy wins. If there is more than one player with the most votes, the vote is redone.

As the game progresses, at each turn, players will receive the following requests. (There is nothing players can do to affect the game other than the requests below, so players will need to play with that in mind.

1. to the player asking the question (for the first turn, Player 1; thereafter, the player who answered the question on the previous turn):
    1. Now it's your turn to ask a question. Choose 1-5 players other than yourself, and write a question.
    Please write your response in the form (n_player,"question")
2. When asking for answers to subsequent questions
    1. In the nth question, player P asked you the following. Please write your answer to this question inside the "" mark.'
3. After all questions and answers have been heard, all players will be asked the following on each turn. Note that players may not interact with each other other other than to name players or not to name players.
    1. At this point, do you want to use the opportunity to name a spy? If yes, write the number of the player you want to name, otherwise write an X in your response.Please only write numbers or Xs in your response.
4. if a player has named someone else as a player, they will be asked to vote as follows. Note that players cannot interact with each other other other than O and X. 1.
    1. 'Subsequently, player P accused player Q of being a spy. Therefore, a vote was held. If you want to hang player Q as a spy, please enter O. If not, please enter X.'
5. At this point, if no one has accused another player this turn, or if a vote is held but does not receive a sufficient number of votes, the game continues and the spy reveals his or her identity and decides whether to guess the location as follows.
    1. At this point, would you identify yourself as a spy and guess the place?
    If you would like to reveal your identity and guess the place, please enter the name of the place. If not, please enter an X. Please only answer with a place name or X.
6. If the spy has not revealed his or her identity at this point, you will move on to the next turn and repeat the sequence from 1.
7. At the end of turn 10, a vote is taken to finally reveal the spy. Note that requests are received in order, starting with Player 1, and may include a three-sentence statement along with the person to be voted out. This vote is repeated until a particular player receives a majority of the votes.
Additional. The location is chosen as one of the following lists (no capitalization)
[airplane, amusement park, bank, beach, carnival, casino, circus tent, corporate party, crusader army, day spa, embassy, hospital, hotel, military base, movie studio, nightclub, ocean liner, passenger train, pirate ship, polar station, police station, restaurant, school, service station, space station, submarine, supermarket, theater, university, zoo]
(Additional) When it comes to strategy and how to play "Spy Fall," the following facts can be helpful.
1. players should basically keep their conversations vague so that the spy doesn't notice the location. You can execute this effectively by framing your responses to correspond to at least 4 or 5, and, depending on the level of ambiguity needed, up to 20~30 different locations.
2. spies will also try to prove that they are not spies by choosing their questions in this way.
3. You'll also want to confuse the spy as much as possible in order to reach your goal before they do. Keep in mind that unlike normal players, spies don't have information about the location. 
4. In response, the Spy will try to deflect suspicion away from themselves by using phrases that imply specific information about the location, or by pointing to other players to add to the confusion.
5. To find the spy through the game, you need to closely observe the questions and answers, as well as the nominations and votes of other players. Observe who is suspecting whom.
6. In the game, players have no way to express their opinions other than by asking questions, answering questions, and calling out spies. Therefore, if you suspect a player, you should immediately call them out as a spy. If this is a reasonable suspicion, the other players will agree, and if not, they will disagree. Calling out spies is an important strategy because it is the only way to know what other players are doing.
7. Spies should reveal their identity when they have enough information about the location, or when they believe the risk of other players identifying them as a spy is sufficiently great.
You are player 1. You're not a spy; the location is '{}'. Now it's your turn to ask a question. Choose 1-5 players other than yourself, and write a question.
 Please write your response in the form (n_player,"question")

'''
testscript

In [ ]:
first_turn_questions_GPT4 = {location:None for location in location_list}
first_turn_questions_GPT35 = {location:None for location in location_list}

## GPT-4

In [ ]:
for location in location_list:
    first_turn_questions_GPT4[location] = q_getGPT4response(testscript.format(location))
    print(location)
    print(first_turn_questions_GPT4[location])
    print('\n\n')

## GPT-3.5-turbo

In [ ]:
for location in location_list:
    first_turn_questions_GPT35[location] = q_getGPT35response(testscript.format(location))
    print(location)
    print(first_turn_questions_GPT35[location])
    print('\n\n')

### cf. If model doesn't respond with (n,player), we've repeated the request.

In [ ]:
first_turn_questions_GPT35_2 = {location:None for location in location_list}
for location in ['amusement park', 'bank', 
                 'casino', 'circus tent', 'corporate party',
                 'crusader army', 'day spa', 'hospital',
                 'military base', 'movie studio', 'nightclub',
                 'ocean liner',  'pirate ship', 
                 'polar station', 
                  'space station', 'submarine', 
                 'theater', 'zoo']:
    first_turn_questions_GPT35_2[location] = q_getGPT35response(testscript.format(location))
    print(location)
    print(first_turn_questions_GPT35_2[location])
    print('\n\n')

In [ ]:



first_turn_questions_GPT35_3 = {location:None for location in location_list}
for location in [
                'corporate party',
                  'hospital', 'zoo']:
    first_turn_questions_GPT35_3[location] = q_getGPT35response(testscript.format(location))
    print(location)
    print(first_turn_questions_GPT35_3[location])
    print('\n\n')

In [ ]:

first_turn_questions_GPT35_4 = {location:None for location in location_list}
for location in [
                  'hospital', 'zoo']:
    first_turn_questions_GPT35_4[location] = q_getGPT35response(testscript.format(location))
    print(location)
    print(first_turn_questions_GPT35_4[location])
    print('\n\n')

In [ ]:
first_turn_questions_GPT35_5 = {location:None for location in location_list}
for location in [
                  'hospital', 'zoo']:
    first_turn_questions_GPT35_5[location] = q_getGPT35response(testscript.format(location))
    print(location)
    print(first_turn_questions_GPT35_5[location])
    print('\n\n')

In [ ]:
first_turn_questions_GPT35_6= {location:None for location in location_list}
for location in [
                  'hospital', 'zoo']:
    first_turn_questions_GPT35_6[location] = q_getGPT35response(testscript.format(location))
    print(location)
    print(first_turn_questions_GPT35_6[location])
    print('\n\n')

In [ ]:
first_turn_questions_GPT35_7 = {location:None for location in location_list}
for location in ['zoo']:
    first_turn_questions_GPT35_7[location] = q_getGPT35response(testscript.format(location))
    print(location)
    print(first_turn_questions_GPT35_7[location])
    print('\n\n')

In [ ]:
first_turn_questions_GPT35_8 = {location:None for location in location_list}
for location in ['zoo']:
    first_turn_questions_GPT35_8[location] = q_getGPT35response(testscript.format(location))
    print(location)
    print(first_turn_questions_GPT35_8[location])
    print('\n\n')

In [ ]:
first_turn_questions_GPT35_9 = {location:None for location in location_list}
for location in ['zoo']:
    first_turn_questions_GPT35_9[location] = q_getGPT35response(testscript.format(location))
    print(location)
    print(first_turn_questions_GPT35_9[location])
    print('\n\n')

In [ ]:
first_turn_questions_GPT35_10 = {location:None for location in location_list}
for location in ['zoo']:
    first_turn_questions_GPT35_10[location] = q_getGPT35response(testscript.format(location))
    print(location)
    print(first_turn_questions_GPT35_10[location])
    print('\n\n')

In [ ]:
first_turn_questions_GPT35_11 = {location:None for location in location_list}
for location in ['zoo']:
    first_turn_questions_GPT35_11[location] = q_getGPT35response(testscript.format(location))
    print(location)
    print(first_turn_questions_GPT35_11[location])
    print('\n\n')

In [ ]:
first_turn_questions_GPT35_12 = {location:None for location in location_list}
for location in ['zoo']:
    first_turn_questions_GPT35_12[location] = q_getGPT35response(testscript.format(location))
    print(location)
    print(first_turn_questions_GPT35_12[location])
    print('\n\n')

### Requesting police station for gpt-3.5-turbo - (we've omitted.)

In [ ]:
first_turn_questions_GPT35_12 = {location:None for location in location_list}
for location in ['police station']:
    first_turn_questions_GPT35_12[location] = q_getGPT35response(testscript.format(location))
    print(location)
    print(first_turn_questions_GPT35_12[location])
    print('\n\n')